In [1]:
!pip install pathway bokeh panel --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [11]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/dataset.csv')

vehicle_weights = {'car': 1.0, 'bike': 0.5, 'truck': 1.5, 'cycle': 0.3}
df['VehicleTypeWeight'] = df['VehicleType'].map(vehicle_weights) if 'VehicleType' in df.columns else 1.0

traffic_map = {'low': 0.2, 'average': 0.5, 'high': 1.0}
df['TrafficScore'] = df['TrafficConditionNearby'].map(traffic_map).fillna(0.4)

df['Timestamp'] = pd.to_datetime(
    df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
    format='%d-%m-%Y %H:%M:%S'
)

df = df.rename(columns={'SystemCodeNumber': 'ParkingLotID'})
df = df[['ParkingLotID', 'Timestamp', 'Occupancy', 'Capacity',
         'QueueLength', 'TrafficScore', 'IsSpecialDay', 'VehicleTypeWeight']]

df.to_csv('parking_stream.csv', index=False)


In [17]:
import pathway as pw
import datetime

# Define schema
class ParkingSchema(pw.Schema):
    ParkingLotID: str
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficScore: float
    IsSpecialDay: int
    VehicleTypeWeight: float

# Load stream data
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=100)

# Parse timestamp and group by day
fmt = "%Y-%m-%d %H:%M:%S"
data_ts = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

# Pricing parameters
alpha, beta, gamma, delta, epsilon = 1.2, 0.8, 0.5, 1.0, 0.6
base_price = 10
lambda_ = 0.6

# Group into 1-hour tumbling windows and reduce
windowed = data_ts.windowby(
    pw.this.t,
    instance=(pw.this.ParkingLotID, pw.this.day),
    window=pw.temporal.tumbling(datetime.timedelta(hours=1)),
    behavior=pw.temporal.exactly_once_behavior()
).reduce(
    t = pw.this._pw_window_end,
    occ_sum = pw.reducers.sum(pw.this.Occupancy),
    cap = pw.reducers.max(pw.this.Capacity),
    queue_sum = pw.reducers.sum(pw.this.QueueLength),
    traffic_sum = pw.reducers.sum(pw.this.TrafficScore),
    special_max = pw.reducers.max(pw.this.IsSpecialDay),
    vehicle_sum = pw.reducers.sum(pw.this.VehicleTypeWeight),
    count = pw.reducers.count()
)

# Step 1: Averages
stage_1 = windowed.with_columns(
    occ = pw.this.occ_sum / pw.this.count,
    queue = pw.this.queue_sum / pw.this.count,
    traffic = pw.this.traffic_sum / pw.this.count,
    vehicle = pw.this.vehicle_sum / pw.this.count,
    special = pw.this.special_max
)

# Step 2: Raw demand calculation
stage_2 = stage_1.with_columns(
    raw_demand = (
        alpha * (pw.this.occ / pw.this.cap) +
        beta * pw.this.queue -
        gamma * pw.this.traffic +
        delta * pw.this.special +
        epsilon * pw.this.vehicle
    )
)

# Step 3: Normalize demand
stage_3 = stage_2.with_columns(
    norm_demand = pw.this.raw_demand / 10
)

# Step 4: Unclipped price
stage_4 = stage_3.with_columns(
    unclipped_price = base_price * (1 + lambda_ * pw.this.norm_demand)
)

# Step 5: Final price with bounds
final = stage_4.with_columns(
    price = pw.apply(lambda x: max(5.0, min(20.0, x)), pw.this.unclipped_price)
)


In [18]:
import panel as pn
import bokeh.plotting

pn.extension()

def plot_price(source):
    fig = bokeh.plotting.figure(
        height=400, width=800,
        title="Real-Time Parking Price",
        x_axis_type="datetime"
    )
    fig.line("t", "price", source=source, line_width=2, color="blue")
    fig.scatter("t", "price", source=source, size=6, color="red")  # updated line
    return fig

viz = final.plot(plot_price, sorting_col="t")
pn.Column(viz).servable()


Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [19]:
%%capture --no-display
pw.run()


Output()